In [1]:
import torch
import numpy as np
from transformers import BertTokenizer
import pandas as pd

/Users/vibha/anaconda3/envs/clare/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/s9/bwgdj6ss6ls2v2hqw4mkb5l80000gn/T/ipykernel_9095/3671335666.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
from BertClassifier import BertClassifier
from train import train_model
from eval import evaluate_model
from utils import load_checkpoint

In [2]:
#Read the synthetically create data into a pandas dataframe
df = pd.read_csv("./clare2.csv")

In [3]:
df

,question,label
0,How do you cope with feelings of anxiety or pa...,1
1,What strategies do you use to manage symptoms ...,1
2,Can you share some techniques for improving se...,1
3,How do you recognize and address triggers that...,1
4,What are some effective ways to communicate wi...,1
...,...,...
463,What's your favorite way to celebrate a person...,5
464,"If you could have any exotic animal as a pet, ...",5
465,Share a moment when you felt completely at pea...,5
466,What's the most memorable gift you've ever giv...,5


In [4]:
#split the data into train,val and test.
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

374 47 47


/Users/vibha/anaconda3/envs/clare/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
#I trained the Model on colab, using a A100 to save time.
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
              
train_model(model, df_train, df_val, LR, EPOCHS)

NameError: name 'train' is not defined

In [15]:
model = load_checkpoint("model.pt",model)

Model loaded from <== model.pt


In [16]:
#function to evaluate the model performance.
def evaluate(model, test_data):
    
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    outlier_list = []
    y_pred = []

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        id = 0
        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)
              
              output = model(input_id, mask)
              if (output.argmax(dim=1)!=test_label):
                outlier_list.append(id)
              y_pred.append(output.argmax(dim=1))
                         

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
              id = id+1
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    return outlier_list,y_pred

In [17]:
outlier_list,y_pred = evaluate_model(model, df_test)

Test Accuracy:  1.000


In [26]:
df_test.iloc[2]['question']

'How do I know if my suicidal thoughts are a result of a mental health condition?'

In [27]:
df_test.iloc[2]

question    How do I know if my suicidal thoughts are a re...
label                                                       4
Name: 319, dtype: object